<a href="https://colab.research.google.com/github/xuwangfmc/dlbook/blob/main/wb_hydra/gHydra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hydra

Hydra是Facebook AI 开发的开源 Python 框架，可简化研究和其他复杂应用程序的开发。这个新框架提供了一种强大的能力，可以从命令行和配置文件中组合和覆盖配置。作为PyTorch 生态系统的一部分，Hydra 帮助 PyTorch 研究人员和开发人员更轻松地管理复杂的机器学习项目。Hydra 是通用的，可以应用于机器学习以外的领域。Hydra 这个名字来自于它能够运行多个类似工作的能力——就像一个有多个头的 Hydra。


![](https://imgur.com/lFLsaiu.png)


主要特点：
可从多个来源组合的分层配置

可以从命令行指定或覆盖配置

动态命令行选项卡补全

在本地运行您的应用程序或启动它以远程运行

使用单个命令运行具有不同参数的多个作业

注：本教程所有代码需要在本地运行

安装 hydra库

pip install hydra-core --upgrade

##1.通过命令行添加设置

创建my_app.py文件



```python
from omegaconf import DictConfig, OmegaConf
import hydra

@hydra.main(config_path=None)
def my_app(cfg: DictConfig) -> None:
    print(OmegaConf.to_yaml(cfg))

if __name__ == "__main__":
    my_app()
```



命令行中输入


```
python my_app.py +db.driver=mysql +db.user=omry +db.password=secret
```

可以观察到控制台打印以下输出：


```yaml
db:
  driver: mysql
  user: omry
  password: secret
```




##2.创建YAML文件进行参数配置


创建config.yaml文件，并添加以下内容：

```yaml
db: 
  driver: mysql
  user: omry
  password: secret
```




改写上面1.的my_app.py文件的部分字段，如下所示：



```python
@hydra.main(config_path=".", config_name="config")
def my_app(cfg):
    print(OmegaConf.to_yaml(cfg))
```



命令行中输入


```
python my_app.py db.user=root db.password=1234
```
控制台会打印命令行参数覆盖后的结果。

用+符号可以添加设置的内容。

```
python my_app.py +db.timeout=5
```






##3.将YAML文件分组并调用



按照以下结构创建文conf与db目录, mysql.yaml和postresql.yaml两个文件。


```
├─ conf
│  └─ db
│      ├─ mysql.yaml
│      └─ postgresql.yaml
└── my_app.py

```

mysql.yaml文件添加以下内容


```yaml
driver: mysql
user: omry
password: secret
```

postgresql.yaml文件添加以下内容


```yaml
driver: postgresql
user: postgres_user
password: drowssap
timeout: 10
```

将2.中的my_app.py文件的部分内容修改如下：


```python
@hydra.main(config_path="conf")
def my_app(cfg: DictConfig) -> None:
    print(OmegaConf.to_yaml(cfg))
```


命令行输入以下指令调用postgresql.yaml文件


```
python my_app.py +db=postgresql
```

此处仍然可以通过命令行设置覆盖原来的参数，如：


```
python my_app.py +db=postgresql db.timeout=20
```







##4.设置config.yaml文件的默认调用情况

在config.yaml文件中运用defaults关键字可以设置默认调用的内容。在conf目录下创建config.yaml文件并添加以下内容：

```yaml
defaults:
  - db: mysql

```

再将3.中的my_app.py文件的部分内容修改如下：


```python
@hydra.main(config_path="conf", config_name="config")
def my_app(cfg: DictConfig) -> None:
    print(OmegaConf.to_yaml(cfg))
```
此时运行my_app.py文件会默认调用mysql.yaml的内容。


```
python my_app.py
```
此时仍可通过命令行调用其它YAML文件和覆盖其中的单个值：


```
python my_app.py db=postgresql db.timeout=20
```
如果config.yaml中与mysql.yaml文件有重复的关键字定义，可以设置_self_关键字进行管理。若想调用config.yaml文件的user值，按如下设置：


```yaml
defaults:
  - db: mysql
  - _self_

db:
  user: root
```
若想调用mysql.yaml文件中的user值，按如下设置：
```yaml
defaults:
  - _self_
  - db: mysql

db:
  user: root
```



也可通过_self_值设置调用config.yaml还是mysql.yaml文件中的值。

##5.组合多个YAML文件进行设置

通过设置config.yaml文件的内容，可以调用组合的YAML文件进行设置。

下载教程中的example文件夹，具体目录布局如下：


```
├── conf
│   ├── config.yaml
│   ├── db
│   │   ├── mysql.yaml
│   │   └── postgresql.yaml
│   ├── schema
│   │   ├── school.yaml
│   │   ├── support.yaml
│   │   └── warehouse.yaml
│   └── ui
│       ├── full.yaml
│       └── view.yaml
└── my_app.py
```
其中config.yaml文件的设置如下：


```yaml
defaults:
  - db: mysql
  - ui: full
  - schema: school
```
命令行中运行my_app.py文件可以打印设置内容。


```
python my_app.py
```

##6.多运行模式

在5.的路径下，使用--multirun(-m)标志并传递一个逗号分隔的列表，指定要扫描的每个维度的值。

```
python my_app.py -m db=mysql,postgresql schema=warehouse,support,school
```
该命令会使用所有参数的组合运行my_app.py文件。
